In [26]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler

In [2]:
dataset_info = pd.read_csv("crop_face_data.csv")
dataset_info

,file_path,gender,age
0,Dataset\Adience_Dataset\Crop_Faces\0.jpg,f,"(25, 32)"
1,Dataset\Adience_Dataset\Crop_Faces\1.jpg,m,"(25, 32)"
2,Dataset\Adience_Dataset\Crop_Faces\2.jpg,f,"(25, 32)"
3,Dataset\Adience_Dataset\Crop_Faces\4.jpg,m,"(25, 32)"
4,Dataset\Adience_Dataset\Crop_Faces\5.jpg,f,"(25, 32)"
...,...,...,...
13541,Dataset\Adience_Dataset\Crop_Faces\17444.jpg,m,"(25, 32)"
13542,Dataset\Adience_Dataset\Crop_Faces\17447.jpg,m,"(25, 32)"
13543,Dataset\Adience_Dataset\Crop_Faces\17448.jpg,f,"(25, 32)"
13544,Dataset\Adience_Dataset\Crop_Faces\17449.jpg,f,"(25, 32)"


In [3]:
data_file_path = dataset_info['file_path'].tolist()
gender = dataset_info['gender'].tolist()
age = dataset_info['age'].tolist()

In [4]:
le_gender = LabelEncoder().fit(gender)
le_gender = 

cat_gender = tf.keras.utils.to_categorical(le_gender, num_classes=2)
cat_gender

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [28]:
BATCH_SIZE = 8

DROP_OUT_RATE = 0.2

In [20]:
def load_image( image_path , label ):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img , label

In [6]:
# Stratified K-Fold
k_fold = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
k_fold

StratifiedKFold(n_splits=5, random_state=12, shuffle=True)

In [27]:
EfficientNet = tf.keras.applications.efficientnet.EfficientNetB0(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [29]:
model= Sequential()
model.add( EfficientNet )
model.add( GlobalAveragePooling2D() ) 
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(128, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(2, activation='softmax') )

In [30]:
initial_learning_rate = 0.01

def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.math.exp(-k*epoch)

In [31]:
lr_scheduler = LearningRateScheduler(lr_exp_decay, verbose=1)

In [32]:
log_dir = os.path.join('Logs')
CHECKPOINT_PATH = os.path.join('CheckPoints')
tb_callback = TensorBoard(log_dir=log_dir)

cp = ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_categorical_accuracy',
                     save_best_only = True,
                     verbose = 1)

In [33]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    #loss='categorical_crossentropy',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=['categorical_accuracy'],
)

In [34]:
for idx , (k_train_index, k_val_index) in enumerate( k_fold.split(data_file_path, gender) ):
    
    train_file_path , train_gender = dataset_info.loc[k_train_index]['file_path'].tolist() , dataset_info.loc[k_train_index]['gender'].tolist()
   
    val_file_path , val_gender = dataset_info.loc[k_val_index]['file_path'].tolist() , dataset_info.loc[k_val_index]['gender'].tolist()
    
    le_gender = LabelEncoder().fit(train_gender)
    train_gender = le_gender.transform(train_gender)
    train_gender = tf.keras.utils.to_categorical(train_gender, num_classes=2)
    
    val_gender = le_gender.transform(val_gender)
    val_gender = tf.keras.utils.to_categorical(val_gender, num_classes=2)
    
    train_dataset = tf.data.Dataset.from_tensor_slices( (train_file_path , train_gender) )
    val_dataset = tf.data.Dataset.from_tensor_slices( (val_file_path , val_gender) )
    
    
    train_dataset = train_dataset.map( load_image, 
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .cache()\
                .prefetch(tf.data.experimental.AUTOTUNE)

    val_dataset = val_dataset.map( load_image, 
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .cache()\
                .prefetch(tf.data.experimental.AUTOTUNE)
    
    print("*"*25 , "Fold : ",idx , "*"*25)
    hist = model.fit(train_dataset,
                 validation_data=val_dataset,
                 callbacks=[lr_scheduler , cp , tb_callback],
                 epochs = 20,
                 verbose = 1 
                )

************************* Fold :  0
Epoch 1/20

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


 153/1355 [==>...........................] - ETA: 5:33 - loss: 0.8892 - categorical_accuracy: 0.5539

KeyboardInterrupt: 